# Import thư viện cần thiết

In [2]:
import json
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve


In [3]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from scipy.sparse import csr_matrix, save_npz, load_npz

# Load file json

In [4]:
def load_json_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read().strip()
        try:
            data = json.loads(raw)
            if isinstance(data, dict):
                data = [data]
        except json.JSONDecodeError:
            data = [json.loads(line) for line in raw.splitlines() if line.strip()]
    return pd.DataFrame(data)

PATH = "data/processed_data/processed_data.json"
df = load_json_file(PATH)


In [5]:
df.head()

,url,url_img,title,description,content,metadata,title_clean,desc_clean,content_clean
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,"{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,"{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...


In [6]:
meta_df = pd.json_normalize(df['metadata'])
meta_df.head(20)

,cat,subcat,published_date,author
0,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown
5,sức khỏe,Tin tức,24/07/2024 - 08:49,Võ Thu
6,sức khỏe,Tin tức,25/07/2024 - 08:43,Võ Thu
7,sức khỏe,Tin tức,07/09/2024 - 16:38,Võ Thu
8,sức khỏe,Tin tức,05/09/2024 - 10:41,Võ Thu
9,sức khỏe,Tin tức,04/09/2024 - 10:36,Võ Thu


In [7]:
col = meta_df['subcat']

cnt_none = (col.apply(lambda x: x is None)).sum()
cnt_nan  = col.isna().sum()
cnt_empty = (col.astype(str).str.strip() == "").sum()   

print("None (is None):", cnt_none)
print("NaN (isna):    ", cnt_nan)
print('Empty "":      ', cnt_empty)

total_blank = ((col.isna()) | (col.astype(str).str.strip() == "")).sum()
print("Total blank:", total_blank)


None (is None): 318
NaN (isna):     318
Empty "":       3753
Total blank: 4071


In [8]:
meta_df['subcat'] = meta_df['subcat'].replace({
    "": np.nan,
    " ": np.nan,
    "nan": np.nan,
    "None": np.nan
})

# Nếu có giá trị None của Python thì pandas.isna() sẽ bắt được
meta_df['subcat'] = meta_df['subcat'].where(~meta_df['subcat'].isna(), np.nan)

# Kiểm tra lại
print(meta_df['subcat'].value_counts(dropna=False))
print("Tổng số NaN:", meta_df['subcat'].isna().sum())


subcat
NaN                     4882
Tin tức                  393
Tuyển sinh               161
Quân sự                   94
Thị trường                87
                        ... 
Việt Nam và thế giới       1
Coaching                   1
Cộng đồng                  1
Bảo hiểm                   1
Tâm sự                     1
Name: count, Length: 130, dtype: int64
Tổng số NaN: 4882


In [9]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8258 entries, 0 to 8257
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cat             8258 non-null   object
 1   subcat          3376 non-null   object
 2   published_date  8258 non-null   object
 3   author          8258 non-null   object
dtypes: object(4)
memory usage: 258.2+ KB


In [10]:
df_merge = pd.concat([df.drop(columns=['metadata']), meta_df], axis=1)

In [11]:
df_merge.head()

,url,url_img,title,description,content,title_clean,desc_clean,content_clean,cat,subcat,published_date,author
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown


In [12]:
print(df_merge['cat'].unique())
df_merge['cat'].value_counts()


['sức khỏe' 'thể thao' 'văn hóa - giải trí' 'xã hội' 'du lịch' 'chính trị'
 'pháp luật' 'thế giới' 'khoa học - công nghệ' 'giáo dục' 'kinh tế']


cat
xã hội                  874
sức khỏe                854
văn hóa - giải trí      801
thể thao                775
kinh tế                 765
giáo dục                756
thế giới                752
du lịch                 733
pháp luật               695
khoa học - công nghệ    652
chính trị               601
Name: count, dtype: int64

In [13]:
df_merge.head()

,url,url_img,title,description,content,title_clean,desc_clean,content_clean,cat,subcat,published_date,author
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown


In [14]:
df_new = df_merge[['title_clean', 'desc_clean', 'content_clean', 'cat']].copy()
df_new.head()

,title_clean,desc_clean,content_clean,cat
0,trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe
1,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe
2,hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe
3,chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe
4,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe


# Xử lý dữ liệu TF-IDF

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

df_new["text"] = (df_new["title_clean"] + " " +
                  df_new["desc_clean"]  + " " +
                  df_new["content_clean"]).astype(str)

X = df_new["text"]
y = df_new["cat"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

vectorizer = TfidfVectorizer(
    preprocessor=lambda x: x,   # giữ nguyên text
    tokenizer=str.split,        # tách theo khoảng trắng
    token_pattern=None,         # vô hiệu hóa regex mặc định
    ngram_range=(1,2),          # unigram + bigram
    min_df=2,
    max_df=0.9
)

Xtr = vectorizer.fit_transform(X_train)
Xte = vectorizer.transform(X_test)

print("Train shape:", Xtr.shape, " Test shape:", Xte.shape)
print("10 feature đầu tiên:", vectorizer.get_feature_names_out()[:10])


Train shape: (6606, 81747)  Test shape: (1652, 81747)
10 feature đầu tiên: ['00' '000' '0000' '00096' '000_binh_sĩ' '000_btc' '000_bản' '000_bảng'
 '000_bộ' '000_ca']


## Save tex

In [48]:
OUT_DIR = "./vocab_output"
os.makedirs(OUT_DIR, exist_ok=True)
VOC_TEX = os.path.join(OUT_DIR, "vocabulary.tex")
VOC_CSV = os.path.join(OUT_DIR, "vocabulary.csv")  

In [50]:
assert set(['title_clean','desc_clean','content_clean','cat']).issubset(df_new.columns)

In [51]:
X = df_new[['title_clean','desc_clean','content_clean']]
y = df_new['cat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train size:", X_train.shape[0])
print("Test size: ", X_test.shape[0])


Train size: 6606
Test size:  1652


In [52]:
tfidf_title = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=2, max_df=0.9)
tfidf_desc  = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=2, max_df=0.9)
tfidf_cont  = TfidfVectorizer(max_features=8000, ngram_range=(1,2), min_df=3, max_df=0.9)

ct = ColumnTransformer(
    transformers=[
        ("title",   tfidf_title, "title_clean"),
        ("desc",    tfidf_desc,  "desc_clean"),
        ("content", tfidf_cont,  "content_clean"),
    ],
    remainder="drop",
    verbose=False
)

In [53]:
_ = ct.fit_transform(X_train)
vec_title_fitted = ct.named_transformers_['title']
vec_desc_fitted  = ct.named_transformers_['desc']
vec_cont_fitted  = ct.named_transformers_['content']



In [54]:
def terms_idf_df(vec: TfidfVectorizer, prefix: str) -> pd.DataFrame:
    return pd.DataFrame({
        "feature": [f"{prefix}__{t}" for t in vec.get_feature_names_out()],
        "source": prefix,
        "term":   vec.get_feature_names_out(),
        "idf":    vec.idf_,
    })

# tạo bảng vocab từ CÁC VEC ĐÃ FIT
v_title = terms_idf_df(vec_title_fitted, "title")
v_desc  = terms_idf_df(vec_desc_fitted,  "desc")
v_cont  = terms_idf_df(vec_cont_fitted,  "content")
vocab_df = pd.concat([v_title, v_desc, v_cont], ignore_index=True)

# Sắp theo đúng thứ tự cột của ma trận (nếu bạn cần đồng bộ hoá)
feature_order = ct.get_feature_names_out()
vocab_df = vocab_df.set_index("feature").loc[feature_order].reset_index()

# (tuỳ chọn) sắp theo idf giảm dần
vocab_sorted = vocab_df.sort_values("idf", ascending=False).reset_index(drop=True)

# Lưu CSV (tiện dùng code)
vocab_sorted.to_csv(VOC_CSV, index=False)

# Xuất LaTeX
def latex_escape(s: str) -> str:
    return (s.replace('\\','\\textbackslash ')
             .replace('_','\\_').replace('&','\\&').replace('%','\\%')
             .replace('#','\\#').replace('{','\\{').replace('}','\\}')
             .replace('$','\\$'))

with open(VOC_TEX, "w", encoding="utf-8") as f:
    f.write("\\documentclass{article}\n\\usepackage[utf8]{vietnam}\n\\usepackage{booktabs}\n"
            "\\usepackage{geometry}\n\\geometry{margin=1in}\n\\begin{document}\n"
            "\\section*{Vocabulary TF--IDF}\n"
            "\\noindent Nguồn (title/desc/content), từ/cụm từ và trọng số IDF được fit trên tập train.\\\\\n"
            "\\begin{tabular}{lll}\n\\toprule\nNguồn & Từ/Cụm từ & IDF\\\\\n\\midrule\n")
    LIMIT = 10000
    for _, r in vocab_sorted.head(LIMIT).iterrows():
        f.write(f"{r['source']} & {latex_escape(r['term'])} & {r['idf']:.4f}\\\\\n")
    f.write("\\bottomrule\n\\end{tabular}\n\\end{document}\n")

print(f"Saved LaTeX vocabulary to: {VOC_TEX}")
print(f"Saved vocabulary CSV to: {VOC_CSV}")


Saved LaTeX vocabulary to: ./vocab_output/vocabulary.tex
Saved vocabulary CSV to: ./vocab_output/vocabulary.csv


# Training

## Linear SVM

In [21]:
Xtr = ct.fit_transform(X_train)
Xte = ct.transform(X_test)

print("Train shape:", Xtr.shape, "| Test shape:", Xte.shape)

NameError: name 'ct' is not defined

In [56]:
# Đếm số lượng từng category trong tập test
print("Số lượng bài báo theo category trong tập test")
print(y_test.value_counts())

Số lượng bài báo theo category trong tập test
cat
xã hội                  175
sức khỏe                171
văn hóa - giải trí      160
thể thao                155
kinh tế                 153
giáo dục                151
thế giới                151
du lịch                 147
pháp luật               139
khoa học - công nghệ    130
chính trị               120
Name: count, dtype: int64


In [57]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# ===== Linear SVM =====
svm_clf = LinearSVC(class_weight='balanced', random_state=42)
svm_clf.fit(Xtr, y_train)
svm_pred = svm_clf.predict(Xte)

print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("F1-macro:", f1_score(y_test, svm_pred, average='macro'))
print("F1-weighted:", f1_score(y_test, svm_pred, average='weighted'))

                      precision    recall  f1-score   support

           chính trị     0.7190    0.7250    0.7220       120
             du lịch     0.8500    0.9252    0.8860       147
            giáo dục     0.9205    0.9205    0.9205       151
khoa học - công nghệ     0.8500    0.7846    0.8160       130
             kinh tế     0.8258    0.8366    0.8312       153
           pháp luật     0.8000    0.8633    0.8304       139
            sức khỏe     0.9138    0.9298    0.9217       171
            thế giới     0.9448    0.9073    0.9257       151
            thể thao     0.9744    0.9806    0.9775       155
  văn hóa - giải trí     0.9068    0.9125    0.9097       160
              xã hội     0.6415    0.5829    0.6108       175

            accuracy                         0.8523      1652
           macro avg     0.8497    0.8517    0.8501      1652
        weighted avg     0.8508    0.8523    0.8510      1652

Confusion matrix:
 [[ 87   1   0   1   3  15   1   1   0   2   9]
 

In [58]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# === 1) Cấu hình CV & thước đo ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = 'f1_macro'

# === 2) Lưới tham số cho LinearSVC ===
# Gồm 2 "nhánh" cấu hình:
#  - penalty L2 (mặc định) với dual=True (nhanh cho D >> N, hợp dữ liệu text)
#  - penalty L1 với dual=False (feature selection nhẹ), chỉ hợp lệ với loss='squared_hinge'
param_grid = [
    {
        "C": [0.1, 0.3, 1, 3, 10],
        "class_weight": [None, "balanced"],
        "loss": ["squared_hinge"],
        "penalty": ["l2"],
        "dual": [True],                  # nhanh & ổn cho TF-IDF (sparse, chiều cao)
        "tol": [1e-4, 1e-5],
        "max_iter": [2000, 5000],
    },
    {
        "C": [0.1, 0.3, 1, 3],
        "class_weight": [None, "balanced"],
        "loss": ["squared_hinge"],
        "penalty": ["l1"],               # chỉ dùng được khi dual=False
        "dual": [False],
        "tol": [1e-4, 1e-5],
        "max_iter": [3000, 7000],
    },
]

base = LinearSVC(random_state=42)

grid = GridSearchCV(
    estimator=base,
    param_grid=param_grid,
    scoring=scorer,
    n_jobs=-1,
    cv=cv,
    verbose=1
)

# === 3) Fit trên TRAIN ===
grid.fit(Xtr, y_train)

print("Best params:", grid.best_params_)
print("Best CV f1_macro:", grid.best_score_)

best_svm: LinearSVC = grid.best_estimator_

# === 4) Đánh giá trên TEST ===
svm_pred = best_svm.predict(Xte)

print("\n=== Linear SVM (tuned) on TEST ===")
print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("F1-macro:", f1_score(y_test, svm_pred, average='macro'))
print("F1-weighted:", f1_score(y_test, svm_pred, average='weighted'))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best params: {'C': 0.3, 'class_weight': 'balanced', 'dual': True, 'loss': 'squared_hinge', 'max_iter': 2000, 'penalty': 'l2', 'tol': 0.0001}
Best CV f1_macro: 0.850833169983623

=== Linear SVM (tuned) on TEST ===
                      precision    recall  f1-score   support

           chính trị     0.7411    0.6917    0.7155       120
             du lịch     0.8519    0.9388    0.8932       147
            giáo dục     0.9156    0.9338    0.9246       151
khoa học - công nghệ     0.8583    0.7923    0.8240       130
             kinh tế     0.8228    0.8497    0.8360       153
           pháp luật     0.7885    0.8849    0.8339       139
            sức khỏe     0.9143    0.9357    0.9249       171
            thế giới     0.9580    0.9073    0.9320       151
            thể thao     0.9682    0.9806    0.9744       155
  văn hóa - giải trí     0.9068    0.9125    0.9097       160
              xã hội     0.6623    0.5829 

## SVM

In [24]:
print("- Training ...")
print("- Train size = {}".format(X_train.shape))
model = svm.SVC(kernel='linear', C=1)
model.fit(Xtr, y_train)
print("- model - train complete")


- Training ...
- Train size = (6606,)
- model - train complete


In [25]:
from sklearn.metrics import accuracy_score
print("- Testing ...")
y_test_pred = model.predict(Xte)
y_train_pred = model.predict(Xtr)
print("- Test Acc = {}".format(accuracy_score(y_test, y_test_pred)))
print("- Train Acc = {}".format(accuracy_score(y_train, y_train_pred)))

- Testing ...
- Test Acc = 0.851089588377724
- Train Acc = 0.9632152588555858


# LTSM

In [61]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Giả sử bạn đã có df_merge với cột 'title_clean','desc_clean','content_clean','cat_mapped'
df_text = (df_merge['title_clean'] + " " + df_merge['desc_clean'] + " " + df_merge['content_clean']).astype(str)
labels = df_merge['cat'].astype(str)

# Encode nhãn sang số
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
y_cat = to_categorical(y)  # one-hot cho softmax

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df_text, y_cat, test_size=0.2, stratify=y, random_state=42)

# Tokenizer
max_words = 20000   # vocab size
max_len = 300       # độ dài chuỗi tối đa
tok = Tokenizer(num_words=max_words, oov_token="<OOV>")
tok.fit_on_texts(X_train)

Xtr_seq = tok.texts_to_sequences(X_train)
Xte_seq = tok.texts_to_sequences(X_test)

Xtr_pad = pad_sequences(Xtr_seq, maxlen=max_len, padding='post')
Xte_pad = pad_sequences(Xte_seq, maxlen=max_len, padding='post')

print("Train shape:", Xtr_pad.shape, y_train.shape)


Train shape: (6606, 300) (6606, 11)


In [62]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

num_classes = y_cat.shape[1]

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [63]:
history = model.fit(
    Xtr_pad, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.1, # lấy 10% tâp train để validationư]
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(Xte_pad, y_test, verbose=0)
print("Test accuracy:", acc)


Epoch 1/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 44s 454ms/step - accuracy: 0.1793 - loss: 2.2852 - val_accuracy: 0.3177 - val_loss: 2.0694
Epoch 2/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 43s 464ms/step - accuracy: 0.3539 - loss: 1.8674 - val_accuracy: 0.4871 - val_loss: 1.5842
Epoch 3/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 45s 486ms/step - accuracy: 0.4932 - loss: 1.5400 - val_accuracy: 0.5219 - val_loss: 1.3972
Epoch 4/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 46s 499ms/step - accuracy: 0.5909 - loss: 1.2739 - val_accuracy: 0.5598 - val_loss: 1.3797
Epoch 5/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 44s 475ms/step - accuracy: 0.6577 - loss: 1.0715 - val_accuracy: 0.5855 - val_loss: 1.3603
Epoch 6/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 44s 477ms/step - accuracy: 0.6992 - loss: 0.9603 - val_accuracy: 0.5371 - val_loss: 1.5094
Epoch 7/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 45s 480ms/step - accuracy: 0.7186 - loss: 0.8908 - val_accuracy: 0.6354 - val_loss: 1.2219
Epoch 8/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 45s 479ms/step - accuracy: 0.7514 - loss: 0.8018 - val_accu

KeyboardInterrupt: 